## AIPI 590 - XAI | Assignment #7
### Explainable Deep Learning
#### Author: Tal Erez
#### Colab Notebook:
[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)]()

### To Run in Colab

Verify you are running on a GPU. On the top right of the screen click the down arrow in between "RAM/Disk" and "Gemini"  -> Change Runtime Type -> T4 GPU -> Save

### Introduction
This notebook demonstrates how to use LIME to generate local explanations for images. The images used come from a small version of the ImageNet library, and the pretrained model used for predictions is ResNet34. For a given label, this will output LIME's explanations for each image of that label in the dataset.

### Install required dependencies and import packages

In [ ]:
!pip install -r requirements.txt

In [23]:
import kagglehub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import soundfile as sf
import tcav.activation_generator as act_gen
import tcav.cav as cav
import tcav.model  as model
import tcav.tcav as tcav
import tcav.utils as utils
import tcav.utils_plot as utils_plot
import tensorflow as tf
import tensorflow_hub as hub
import torch
import torchaudio
import utils

In [17]:
device = None

# Fetching the device that will be used throughout this notebook
if torch.backends.mps.is_available():
    # Ensure that all operations are deterministic on GPU (if used) for reproducibility
    torch.use_deterministic_algorithms(True)

    #set device to use mps
    device = torch.device("mps")
elif torch.cuda.is_available():
    # Ensure that all operations are deterministic on GPU (if used) for reproducibility
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    #set device to use cuda
    device = torch.device("cuda:0")
else:
    #set device to use the cpu
    device = torch.device("cpu")

print("Using device", device)

Using device mps


### Hypothesis
- H0: The VGGish model does not show a significant difference in its ability to learn the difference in audio files pronouncing "zero" compared to audio files pronouncing "1."
- H1: The VGGish model shows a significant difference in its ability to learn the difference in pronounciation of "zero" vs "one."

### Load the Data and Model

In [33]:
# Location of audio files (modify with the path to your audio dataset)
source_dir = './data/'

import whisper

model = whisper.load_model("small")
result = model.transcribe("./data/0/0_01_0.wav")
print(result["text"])

/Users/talerez/Documents/AIPI_590_XAI/ExplainableDeepLearning/venv/lib/python3.9/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpo

RuntimeError: mixed dtype (CPU): expect parameter to have scalar type of Float

### Load The Model

In [4]:
# Model wrapper
model_to_run = 'whisper'

# Name of the parent directory for storing results
project_name = 'tcav_audio'

# Working directory and subdirectories for activations and CAVs
working_dir = '.'
activation_dir = working_dir + '/activations/'
cav_dir = working_dir + '/cavs/'

# Define bottleneck layers for VGGish
bottlenecks = ['embedding'] 

# Create directories if they don't exist
os.makedirs(activation_dir, exist_ok=True)
os.makedirs(working_dir, exist_ok=True)
os.makedirs(cav_dir, exist_ok=True)

# regularizer penalty parameter for linear classifier to get CAVs
alphas = [0.1]

# target class
target = 'guitar'

# concepts for testing
concepts = ["strumming", "picking"]